In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image

In [ ]:
#8 MIN TO RUN
def load_data():
    # Load training images and labels
    train_images = []
    train_labels = []
    train_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/images'
    train_label_file = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/train/labels'
    for filename in os.listdir(train_dir):
        img = Image.open(os.path.join(train_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        train_images.append(img_arr)
        
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(train_label_file, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            train_labels.append(label)

    # Load testing images and labels
    test_images = []
    test_labels = []
    test_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/images'
    test_label_dir = '/content/drive/MyDrive/CW_Folder_UG/CW_Dataset/test/labels'
    for filename in os.listdir(test_dir):
        img = Image.open(os.path.join(test_dir, filename))
        img = img.resize((224, 224)) # Resize to a common size
        img_arr = np.array(img)
        test_images.append(img_arr)
        
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_file_path = os.path.join(test_label_dir, label_filename)
        with open(label_file_path, 'r') as f:
            text = f.read().strip()
            # Map "no_mask" to 0, "mask" to 1, and "improper_mask" to 2
            if text == "1":
                label = 1
            elif text == "0":
                label = 0
            elif text == "2":
                label = 2
            test_labels.append(label)

    # Convert the lists to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    return train_images, test_images, train_labels, test_labels

# Load the dataset
X_train, X_test, y_train, y_test = load_data()

print("Number of training images: ", len(X_train))
print("Number of training labels: ", len(y_train))
print("Number of testing images: ", len(X_test))
print("Number of testing labels: ", len(y_test))

print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

# Print information about the dataset
print("Number of training images: ", X_train.shape[0])
print("Number of training labels: ", y_train.shape[0])
print("Number of testing images: ", X_test.shape[0])
print("Number of testing labels: ", y_test.shape[0])

Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458
Shape of X_train:  (2394, 224, 224, 3)
Shape of y_train:  (2394,)
Shape of X_test:  (458, 224, 224, 3)
Shape of y_test:  (458,)
Number of training images:  2394
Number of training labels:  2394
Number of testing images:  458
Number of testing labels:  458


In [ ]:
#Around 1 min
import numpy as np
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Extract HOG features from training data
hog_train = []
for img in X_train:
    hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), visualize=False, multichannel=True, channel_axis=2)
    hog_train.append(hog_feature)

# Extract HOG features from test data
hog_test = []
for img in X_test:
    hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), visualize=False, multichannel=True, channel_axis=2)
    hog_test.append(hog_feature)
    

# Train KNN classifier with best k value
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn.fit(hog_train, y_train)

# Predict on test data
y_pred = clf_knn.predict(hog_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

# Print f1score, precision, and recall
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

<ipython-input-11-b9bed56d2adc>:11: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), visualize=False, multichannel=True, channel_axis=2)
<ipython-input-11-b9bed56d2adc>:17: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), visualize=False, multichannel=True, channel_axis=2)


Accuracy:  0.8777292576419214
Classification Report:

              precision    recall  f1-score   support

           0       0.65      0.43      0.52        51
           1       0.90      0.97      0.93       388
           2       1.00      0.11      0.19        19

    accuracy                           0.88       458
   macro avg       0.85      0.50      0.55       458
weighted avg       0.87      0.88      0.86       458

Confusion Matrix:

[[ 22  29   0]
 [ 10 378   0]
 [  2  15   2]]


In [ ]:
import joblib
import os

# Define directory path to save the joblib file
directory = '/content/drive/MyDrive/CW_Folder_UG/Models'

# Create directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Define file path to save the joblib file
filepath = os.path.join(directory, 'hog-knn.joblib')

# Dump the object to the joblib file
joblib.dump(clf_knn, filepath)

['/content/drive/MyDrive/CW_Folder_UG/Models/hog-knn.joblib']

In [ ]:
import joblib
import os

# create folder if it doesn't exist
folder_path = os.path.join('content', 'drive', 'MyDrive', 'CW_Folder_UG', 'Models')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# save model in folder
file_path = os.path.join(folder_path, 'hog-knn.joblib')
joblib.dump(clf_knn, file_path)


['content/drive/MyDrive/CW_Folder_UG/Models/hog-knn.joblib']